# Query by observatory name/code

### Curate the IRAF observatory list (made two corrections)
[Retrieve IRAF observatories](http://tdc-www.harvard.edu/iraf/rvsao/bcvcorr/obsdb.html)

In [1]:
# Grabbed from: http://tdc-www.harvard.edu/iraf/rvsao/bcvcorr/obsdb.html
# added altitude=0m to Observatorio Astronomico Nacional, Tonantzintla (altitude was missing),
# corrected Whipple observatory latitude from "31.41.3" to "31:41.3"
# Longitudes here are listed with positive to the west.
# Added aliases

IRAF = '''
observatory = "kpno"
    name = "Kitt Peak National Observatory"
    longitude = 111:36.0
    latitude = 31:57.8
    altitude = 2120.
    timezone = 7
    aliases = ["Kitt Peak", "KPNO", "Kitt Peak National Observatory"]

observatory = "ctio"
    name = "Cerro Tololo Interamerican Observatory"
    longitude = 70.815
    latitude = -30.16527778
    altitude = 2215.
    timezone = 4
    aliases = ["Cerro Tololo Interamerican Observatory", "CTIO", "Cerro Tololo"]

observatory = "eso"
    name = "European Southern Observatory"
    longitude = 70:43.8
    latitude = -29:15.4
    altitude = 2347
    timezone = 4
    aliases = ["European Southern Observatory", "ESO"]

observatory = "lick"
    name = "Lick Observatory"
    longitude = 121:38.2
    latitude = 37:20.6
    altitude = 1290
    timezone = 8
    aliases = ["Lick", "Lick Observatory"]

observatory = "mmt"
    name = "Whipple Observatory"
    longitude = 110:53.1
    latitude = 31:41.3
    altitude = 2608
    timezone = 7
    aliases = ["Multiple Mirror Telescope", "MMT"]

observatory = "flwo"
    name = "Whipple Observatory"
    longitude = 110:52:39
    latitude = 31:40:51.4
    altitude = 2320
    timezone = 7
    aliases = ["Whipple", "Whipple Observatory", "FLWO"]

observatory = "cfht"
    name = "Canada-France-Hawaii Telescope"
    longitude = 155:28.3
    latitude = 19:49.6
    altitude = 4215
    timezone = 10
    aliases = ["Canada-France-Hawaii Telescope", "CFHT"]

observatory = "lapalma"
    name = "Roque de los Muchachos, La Palma"
    longitude = 17:52.8
    latitude = 28:45.5
    altitude = 2327
    timezone = 0
    aliases = ["lapalma", "Roque de los Muchachos"]

observatory = "mso"
    name = "Mt. Stromlo Observatory"
    longitude = 210:58:32.4
    latitude = -35:19:14.34
    altitude = 767
    timezone = -10
    aliases = ["MSO", "Mt. Stromlo Observatory"]

observatory = "sso"
    name = "Siding Spring Observatory"
    longitude = 210:56:19.70
    latitude = -31:16:24.10
    altitude = 1149
    timezone = -10
    aliases = ["Siding Spring Observatory", "sso"]

observatory = "aao"
    name = "Anglo-Australian Observatory"
    longitude = 210:56:2.09
    latitude = -31:16:37.34
    altitude = 1164
    timezone = -10
    aliases = ["AAO", "Anglo-Australian Observatory"]

observatory = "mcdonald"
    name = "McDonald Observatory"
    longitude = 104.0216667
    latitude = 30.6716667
    altitude = 2075
    timezone = 6
    aliases = ["McDonald Observatory", "mcdonald"]

observatory = "lco"
    name = "Las Campanas Observatory"
    longitude = 70:42.1
    latitude = -29:0.2
    altitude = 2282
    timezone = 4
    aliases = ["LCO", "Las Campanas Observatory"]

# Submitted by Alan Koski 1/13/93
observatory = "mtbigelow"
    name = "Catalina Observatory: 61 inch telescope"
    longitude = 110:43.9
    latitude = 32:25.0
    altitude = 2510.
    timezone = 7
    aliases = ["mtbigelow", "Catalina Observatory"]
    
# Revised by Daniel Durand 2/23/93
observatory = "dao"
    name = "Dominion Astrophysical Observatory"
    longitude = 123:25.0
    latitude = 48:31.3
    altitude = 229
    timezone = 8
    aliases = ["DAO", "Dominion Astrophysical Observatory"]

# Submitted by Patrick Vielle 5/4/93
observatory = "spm"
    name = "Observatorio Astronomico Nacional, San Pedro Martir"
    longitude = 115:29:13
    latitude = 31:01:45
    altitude = 2830
    timezone = 7
    aliases = ["Observatorio Astronomico Nacional, San Pedro Martir", "SPM"]

# Submitted by Patrick Vielle 5/4/93
observatory = "tona"
    name = "Observatorio Astronomico Nacional, Tonantzintla"
    longitude = 98:18:50
    latitude = 19:01:58
    altitude = 0
    timezone = 8
    aliases = ["tona", "Observatorio Astronomico Nacional, Tonantzintla"]

# Submitted by Don Hamilton 8/18/93
observatory = "Palomar"
    name = "The Hale Telescope"
    longitude = 116:51:46.80
    latitude = 33:21:21.6
    altitude = 1706.
    timezone = 8
    aliases = ["Palomar", "Hale Telescope"]

# Submitted by Pat Seitzer 10/31/93
observatory = "mdm"
    name = "Michigan-Dartmouth-MIT Observatory"
    longitude = 111:37.0
    latitude = 31:57.0
    altitude = 1938.5
    timezone = 7
    aliases = ["Michigan-Dartmouth-MIT Observatory", "MDM"]

# Submitted by Ignacio Ferrin 9/1/94
observatory = "NOV"
    name = "National Observatory of Venezuela"
    longitude = 70:52.0
    latitude = 8:47.4
    altitude = 3610
    timezone = 4
    aliases = ["National Observatory of Venezuela", "NOV"]

# Submitted by Alan Welty 10/28/94
observatory = "bmo"
    name = "Black Moshannon Observatory"
    longitude = 78:00.3
    latitude = 40:55.3
    altitude = 738
    timezone = 5
    aliases = ["Black Moshannon Observatory", "BMO"]

# Submitted by Biwei JIANG 11/28/95
observatory = "BAO"
    name = "Beijing XingLong Observatory"
    longitude = 242:25.5
    latitude = 40:23.6
    altitude = 950.
    timezone = -8
    aliases = ["BAO", "Beijing XingLong Observatory"]

# From Astronomical Almanac 1996
observatory = "keck"
    name = "W. M. Keck Observatory"
    longitude = 155:28.7
    latitude = 19:49.7
    altitude = 4160
    timezone = 10
    aliases = ["keck", "W. M. Keck Observatory", "Keck Observatory"]

# Padova Astronomical Observatory, Asiago, Italy.
# Submitted by Lina Tomasella 6/11/96
observatory = "ekar"
    name = "Mt. Ekar 182 cm. Telescope"
    longitude = 348:25:07.92
    latitude = 45:50:54.92
    altitude = 1413.69
    timezone = -1
    aliases = ["ekar", "Mt. Ekar 182 cm. Telescope"]

# Submitted by Michael Ledlow 8/8/96
observatory = "apo"
    name = "Apache Point Observatory"
    longitude = 105:49.2
    latitude = 32:46.8
    altitude = 2798.
    timezone = 7 
    aliases = ["APO", "Apache Point Observatory", "Apache Point"]

# Submitted by Michael Ledlow 8/8/96
observatory = "lowell"
    name = "Lowell Observatory"
    longitude = 111:32.1
    latitude = 35:05.8
    altitude = 2198. 
    timezone = 7 
    aliases = ["Lowell Observatory", "Lowell"]

# Submitted by S.G. Bhargavi 8/12/96
observatory = "vbo"
    name = "Vainu Bappu Observatory"
    longitude = 281.1734
    latitude = 12.57666
    altitude = 725. 
    timezone = -5.5
    aliases = ["Vainu Bappu Observatory", "VBO"]
'''

## Parse the observatory list, output to cleaned JSON file

In [5]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import numpy as np
from astropy.time import Time
from astropy.coordinates import (SkyCoord, EarthLocation, Latitude, Longitude, Angle)
import astropy.units as u
from astropy.utils import data
from astropy.io import ascii
from astropy.table import Table
import json

codes = []
names = []
longitudes = []
latitudes = []
altitudes = []
timezones = []
aliases = []

for line in IRAF.splitlines():
    if '=' in line:
        strip_line = line.strip()
        value = line.split('=')[1].strip()

        if strip_line.startswith('observatory'):
            codes.append(value[1:-1])
        elif strip_line.startswith('name'):
            names.append(value[1:-1])
        elif strip_line.startswith('longitude'):
            longitudes.append(-1*Longitude(value, unit=u.deg))
        elif strip_line.startswith('latitude'):
            latitudes.append(Latitude(value, unit=u.deg))
        elif strip_line.startswith('altitude'):
            altitudes.append(float(value))
        elif strip_line.startswith('timezone'):
            timezones.append(float(value))
        elif strip_line.startswith('aliases'):
            aliases.append(eval(value))

longitudes = Longitude(longitudes)
latitudes = Latitude(latitudes)
altitudes = np.array(altitudes)*u.m

tbl = Table([codes, names, longitudes, latitudes, altitudes], 
            names=('Code', 'Name', 'Longitude', 'Latitude', 'Altitude'), 
            dtype=(str, str, float, float, float))

output_dict = {}
angle_to_string = lambda theta: Angle(theta).to_string(unit=u.deg)
for i, code in enumerate(tbl['Code']):
    output_dict[code] = dict(name=tbl['Name'][i], 
                             longitude=angle_to_string(tbl['Longitude'].quantity[i]),
                             latitude=angle_to_string(tbl['Latitude'].quantity[i]),
                             elevation_meters=int(tbl['Altitude'][i]),
                             aliases=aliases[i],
                             source='IRAF Observatory Database')

output_json_path = 'observatories.json'
with open(output_json_path, 'w') as output_file:
    output_file.write(json.dumps(output_dict, indent=4, sort_keys=True))

### Inspect JSON output:

In [6]:
%%bash 
more -10 observatories.json

{
    "BAO": {
        "aliases": [
            "BAO", 
            "Beijing XingLong Observatory"
        ], 
        "elevation_meters": 950, 
        "latitude": "40d23m36s", 
        "longitude": "117d34m30s", 
        "name": "Beijing XingLong Observatory", 
        "source": "IRAF Observatory Database"
    }, 
    "NOV": {
        "aliases": [
            "National Observatory of Venezuela", 
            "NOV"
        ], 
        "elevation_meters": 3610, 
        "latitude": "8d47m24s", 
        "longitude": "289d08m00s", 
        "name": "National Observatory of Venezuela", 
        "source": "IRAF Observatory Database"
    }, 
    "Palomar": {
        "aliases": [
            "Palomar", 
            "Hale Telescope"
        ], 
        "elevation_meters": 1706, 
        "latitude": "33d21m21.6s", 
        "longitude": "243d08m13.2s", 
        "name": "The Hale Telescope", 
        "source": "IRAF Observatory Database"
    }, 
    "aao": {
        "aliases": [
            "AAO"

## Demonstrate loading the table, retrieving values

Sanity checks for positive longitudes to the East via Google search:

| Obs | Longitude [deg] | Latitude [deg]| 
|-----|-----------|---------|
|Keck | -155.4750 | 19.8264 |
|CTIO | -70.8065  |-30.1697 | 
| APO | -105.8203 | 32.7803 |


In [7]:
observatory_db = json.loads(open(output_json_path).read())

def from_code(code, db=observatory_db):
    return EarthLocation.from_geodetic(db[code]['longitude'], 
                                       db[code]['latitude'], 
                                       db[code]['elevation_meters'])

print(from_code('keck').to_geodetic())
print(from_code('ctio').to_geodetic())
print(from_code('apo').to_geodetic())

(<Longitude -155.47833333333332 deg>, <Latitude 19.828333333333326 deg>, <Quantity 4160.000000000752 m>)
(<Longitude -70.815 deg>, <Latitude -30.165277777777778 deg>, <Quantity 2214.9999999993697 m>)
(<Longitude -105.82000000000002 deg>, <Latitude 32.78000000000001 deg>, <Quantity 2797.9999999996007 m>)
